In [ ]:
#!apt-get update
#!apt-get install -y swig python3-dev

In [ ]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
!pip install -r requirements.txt

In [ ]:
# %% [markdown]
# # PPO en CarRacing-v3 con TorchRL (con corrección en make_torchrl_env)
#
# Este Notebook muestra cómo entrenar CarRacing-v3 con PPO usando únicamente TorchRL (sin escribir PPO a mano),
# e incluye gráficos en vivo y una barra de progreso (`tqdm`) que muestra “Avg Reward (Train)” y “Avg Reward (Eval)”.
#
# **Dependencias**:
# ```
# pip install torch torchvision torchrl gymnasium[box2d] tqdm matplotlib
# ```
#
# ———————————————————————————————————————————————————————————————————————————————

# %% [markdown]
# ## 1. Importaciones y utilidades de graficado
#
# Importamos Torch, TorchRL, Gym, Matplotlib, `tqdm`, y definimos la función `plot(logs)` para reconstruir la figura en cada iteración.

# %% [code]
import warnings
warnings.filterwarnings("ignore")
import os
import uuid

import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gymnasium as gym
import numpy as np

from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from tqdm import tqdm

# TorchRL
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.transforms import ToTensorImage, Compose
from torchrl.collectors import SyncDataCollector
from torchrl.envs import AutoResetEnv
from torchrl.envs.utils import ExplorationType, set_exploration_type

from torchrl.modules import SafeModule, ProbabilisticActor, ValueOperator
from torchrl.objectives import ClipPPOLoss
from torchrl.objectives.value import GAE

# — Función de graficado (idéntica a la del IPYNB de ejemplo) —
def plot(logs):
    """
    Recibe un dict `logs` con listas en:
      logs["train_reward"] = [r1, r2, r3, ...]
      logs["eval_reward"]  = [e1, e2, e3, ...]
    y dibuja dos subplots: 
      · Avg Reward (Train)
      · Avg Reward (Eval)
    Llamar esta función tras actualizar `logs` para redibujar la gráfica en línea.
    """
    clear_output(wait=True)
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    axes = axes.flatten()
    titles = ["Avg Reward (Train)", "Avg Reward (Eval)"]
    data = [
        (logs["train_reward"], "blue"),
        (logs["eval_reward"],  "green"),
    ]
    for ax, title, (y, color) in zip(axes, titles, data):
        ax.plot(y, color=color)
        ax.set_title(title)
        ax.relim()
        ax.autoscale_view()
    plt.tight_layout()
    display(fig)
    plt.close(fig)

# %% [markdown]
# ## 2. Wrapper discreto para CarRacing-v3
#
# El entorno original de CarRacing usa un espacio continuo de 3 dimensiones (`[steer, gas, brake]`),
# pero queremos usar PPO discreto con 5 acciones:
#
# 1. `0`: no-op            → `[ 0.0,  0.0,  0.0 ]`  
# 2. `1`: acelerar        → `[ 0.0,  1.0,  0.0 ]`  
# 3. `2`: frenar          → `[ 0.0,  0.0,  0.8 ]`  
# 4. `3`: girar izquierda → `[−1.0,  0.0,  0.0 ]`  
# 5. `4`: girar derecha   → `[ 1.0,  0.0,  0.0 ]`
#
# Este wrapper convierte un entero discreto (0–4) en el vector contínuo que CarRacing espera.

# %% [code]
class DiscreteCarRacingWrapper(gym.ActionWrapper):
    """
    Convierte la acción discreta en un vector [steer, gas, brake].
    """
    def __init__(self, env: gym.Env):
        super().__init__(env)
        self.discrete_actions = [
            np.array([ 0.0,  0.0,  0.0 ], dtype=np.float32),  # 0: no-op
            np.array([ 0.0,  1.0,  0.0 ], dtype=np.float32),  # 1: acelerar
            np.array([ 0.0,  0.0,  0.8 ], dtype=np.float32),  # 2: frenar
            np.array([-1.0,  0.0,  0.0 ], dtype=np.float32),  # 3: girar izquierda
            np.array([ 1.0,  0.0,  0.0 ], dtype=np.float32),  # 4: girar derecha
        ]
        self.action_space = gym.spaces.Discrete(len(self.discrete_actions))

    def action(self, action_discrete: int) -> np.ndarray:
        return self.discrete_actions[int(action_discrete)]

# %% [markdown]
# ## 3. Corrección en make_torchrl_env
#
# Ahora usamos `env_ctor` para que `GymEnv` construya internamente el entorno ya envuelto,
# y semillamos **antes** de envolver, evitando el error de `seed`.

# %% [code]
def make_torchrl_env(seed: int = 0, device: str = "cpu"):
    """
    Crea un GymEnv de TorchRL para CarRacing-v3 con acciones discretas.
    Usamos `env_ctor=` para que GymEnv construya internamente el entorno ya envuelto.
    """
    # 1) Defino la función que al llamarla crea y semilla el entorno base
    def _make_wrapped_env():
        base_env = gym.make("CarRacing-v3", render_mode=None)
        # Semilleo directamente el entorno base
        base_env.reset(seed=seed)
        # Aplico el wrapper discreto
        wrapped = DiscreteCarRacingWrapper(base_env)
        return wrapped

    # 2) Construyo el GymEnv usando `env_ctor` (la función que hace _make_wrapped_env)
    trl_env = GymEnv(
        env_ctor=_make_wrapped_env,
        from_pixels=True,
        pixels_only=True,
        device=device
    )
    # 3) Aplico la transformación de píxeles: uint8 H×W×C → float32 C×H×W en [0,1]
    trl_env.set_transform(Compose(ToTensorImage()))
    return trl_env

# %% [markdown]
# ## 4. Definir Actor–Crítico con SafeModule + ProbabilisticActor + ValueOperator
#
# - La red convolucional: 3 → 32 conv8×8/stride 4 → 64 conv4×4/stride 2 → 64 conv3×3/stride 1 → Flatten → FC 512.  
# - **Policy head**: saca logits sobre 5 acciones discretas.  
# - **Value head**: saca un escalar (valor del estado).  

# %% [code]
# 4.1) Red CNN base
class CNNBody(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)   # → [32,23,23]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)  # → [64,10,10]
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)  # → [64,8,8]
        self.flatten_dim = 64 * 8 * 8
        self.fc = nn.Linear(self.flatten_dim, 512)

    def forward(self, obs_image: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(obs_image))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(-1, self.flatten_dim)
        x = F.relu(self.fc(x))
        return x  # → [B,512]

# 4.2) Construcción del módulo completo con dos salidas
class ActorCriticNet(nn.Module):
    def __init__(self, num_actions: int = 5):
        super().__init__()
        self.body = CNNBody()
        # Cabeza de política: produce logits [B,5]
        self.policy_head = nn.Linear(512, num_actions)
        # Cabeza de valor: produce valor escalar [B,1]
        self.value_head  = nn.Linear(512, 1)

    def forward(self, obs_image: torch.Tensor):
        features = self.body(obs_image)                    # [B,512]
        logits   = self.policy_head(features)              # [B,5]
        value    = self.value_head(features).squeeze(-1)   # [B]
        return logits, value

# 4.3) Armado de SafeModule + ProbabilisticActor + ValueOperator
def make_ac_modules(device: str = "cpu"):
    """
    Devuelve:
      - actor_module: ProbabilisticActor que espera 'pixels' y saca 'action', 'log_prob'
      - value_module: ValueOperator que espera 'pixels' y saca 'state_value'
    """
    base_net = ActorCriticNet(num_actions=5).to(device)
    # Política
    actor_sm = SafeModule(
        module=base_net,
        in_keys=["pixels"],            # recibe tensor 'pixels'
        out_keys=["logits", "value"],  # forward devuelve (logits, value)
        function=lambda m, x: m(x)     # m(x) → (logits,value)
    )
    actor_module = ProbabilisticActor(
        module=actor_sm,
        in_keys=["logits"],
        out_keys=["action", "log_prob"],
        distribution_class=Categorical,
        distribution_kwargs={},       # Categorical(logits=logits)
        return_log_prob=True,
        default_interaction_mode="random",  # en train samplea, en eval hará mode="mode"
    )

    # Valor
    # SafeModule que toma (logits, value) y extrae value para ValueOperator
    value_sm = SafeModule(
        module=base_net,
        in_keys=["pixels"],
        out_keys=["state_value"], 
        function=lambda m, x: m(x)[1].unsqueeze(-1)  
        # m(x)[1] es [B], lo llevamos a [B,1] para ValueOperator
    )
    value_module = ValueOperator(
        in_keys=["state_value"],   # recibe 'state_value'
        out_keys=["state_value"],  # devuelve también 'state_value'
    )

    return actor_module, value_module

# %% [markdown]
# ## 5. Configuración del Collector y pérdida PPO (ClipPPOLoss + GAE)
#
# - `frames_per_batch`: cuántos pasos (frames) recolectamos antes de hacer update de PPO.  
# - `num_iterations`: cuántas iteraciones (épocas) de PPO.  
# - Internamente, `ClipPPOLoss` calculará la pérdida de crítico (MSE), la pérdida de política con clipping, y entropía.  
# - Para GAE definimos un objeto `GAE(gamma, lam)`.

# %% [code]
# 5.1) Hiperparámetros
frames_per_batch   = 1024    # pasos a recolectar por iteración
num_iterations     = 256     # cuántas iteraciones (épocas) de PPO
ppo_epoch          = 4       # cuántas pasadas sobre cada batch de datos
sub_batch_size     = 128     # minibatch para optimizador
gamma              = 0.99
lam                = 0.95
learning_rate      = 2e-4

# 5.2) Crear entornos
device = "cuda" if torch.cuda.is_available() else "cpu"
env_train = make_torchrl_env(seed=123, device=device)
env_eval  = make_torchrl_env(seed=456, device=device)

# 5.3) Crear módulos Actor y Crítico
actor_module, value_module = make_ac_modules(device=device)

# 5.4) Establecer exploración por defecto (importante para PPO “on-policy”)
set_exploration_type(ExplorationType.RANDOM)

# 5.5) ClipPPOLoss y GAE
clip_ppo_loss = ClipPPOLoss(
    actor=actor_module,
    value_network=value_module,
    clip_eps=0.2,
    value_loss_coeff=0.5,
    entropy_coeff=0.01,
    gamma=gamma,
    lambda_=lam,
    max_grad_norm=0.5,
)

# 5.6) Optimizador para actor + crítico
optimizer = torch.optim.Adam(
    list(actor_module.parameters()) + list(value_module.parameters()),
    lr=learning_rate
)

# 5.7) GAE para calcular ventajas
gae_module = GAE(
    gamma=gamma,
    lambda_=lam,
    reduction="mean"
)

# 5.8) Collector síncrono con AutoResetEnv para recolección de transiciones
train_collector = SyncDataCollector(
    env=AutoResetEnv(env_train),
    policy=actor_module,
    frames_per_batch=frames_per_batch,
    total_frames=None,
    device=device,
)

# %% [markdown]
# ## 6. Rutina de entrenamiento + evaluación con tqdm y plot(logs)
#
# En cada iteración (de `0` a `num_iterations-1`):
# 1. Recolectar `frames_per_batch` pasos con `train_collector`.  
# 2. Calcular ventajas y retornos con `gae_module`.  
# 3. Llamar a `clip_ppo_loss` para optimizar política + crítico.  
# 4. Calcular `avg_train_reward` de ese batch.  
# 5. Ejecutar `num_eval_episodes` episodios en `env_eval` para `avg_eval_reward`.  
# 6. Actualizar la barra de progreso (`tqdm`) y guardar métricas en `logs`.  
# 7. Llamar a `plot(logs)` para redibujar en línea.

# %% [code]
# 6.1) Variables para logs
logs = {
    "train_reward": [],
    "eval_reward":  []
}

num_eval_episodes = 3  # cuántos episodios de evaluación por iteración

# Barra de progreso
pbar = tqdm(range(num_iterations), desc="PPO Entrenando", unit="iter")

for iter_idx in pbar:
    # —————————————————————————————————————————————————
    # 1) RECOLECTAR frames_per_batch pasos (rollout on-policy)
    # —————————————————————————————————————————————————
    experience = next(train_collector)
    # experience es un TensorDict con keys:
    #   "pixels", "action", "log_prob", "state_value", "next_pixels", "reward", "done", "rollout_info", ...

    # 2) CALCULAR VENTAJAS Y RETORNOS (GAE)
    # GAE rellena "advantage" y "return" en el mismo TensorDict.
    _ = gae_module(experience)

    # 3) OPTIMIZAR CON ClipPPOLoss
    # Le pasamos `experience`, `optimizer`, `ppo_epoch`, `sub_batch_size`
    loss_ppo = clip_ppo_loss(
        experience,
        optimizer=optimizer,
        ppo_epoch=ppo_epoch,
        mini_batch_size=sub_batch_size,
    )

    # 4) CALCULAR avg_train_reward para este batch 
    # TorchRL almacena en "rollout_info" → "reward_sum" la suma de rewards por episodio.
    train_episode_rewards = experience.get(("rollout_info", "reward_sum")).cpu().numpy()
    avg_train_reward = float(np.mean(train_episode_rewards))
    logs["train_reward"].append(avg_train_reward)

    # —————————————————————————————————————————————————
    # 5) EVALUACIÓN (política determinista)
    # —————————————————————————————————————————————————
    set_exploration_type(ExplorationType.MODE)  # fuerza argmax en ProbabilisticActor
    eval_episode_rewards = []
    for _ in range(num_eval_episodes):
        td = env_eval.reset()  # td contiene "pixels"
        done = False
        ep_rew = 0.0
        while not done:
            obs_td = td.clone().to(device)
            out = actor_module(obs_td)
            action = out.get("action").item()
            td, reward, term, trunc, _ = env_eval.step(action)
            done = bool(term or trunc)
            ep_rew += reward
        eval_episode_rewards.append(ep_rew)
    avg_eval_reward = float(np.mean(eval_episode_rewards))
    logs["eval_reward"].append(avg_eval_reward)
    set_exploration_type(ExplorationType.RANDOM)

    # 6) ACTUALIZAR tqdm Y GRAficar
    pbar.set_postfix({
        "avg_train": f"{avg_train_reward:.1f}",
        "avg_eval":  f"{avg_eval_reward:.1f}"
    })
    plot(logs)

# Cerrar barra
pbar.close()

# %% [markdown]
# ## 7. Guardar modelo final
#
# Si quieres salvar el checkpoint del actor y el crítico, puedes hacer:
# ```python
# save_name = f"ppo_cr_{uuid.uuid4().hex[:6]}.pt"
# checkpoint = {
#     "actor_state_dict": actor_module.state_dict(),
#     "value_state_dict": value_module.state_dict(),
# }
# torch.save(checkpoint, save_name)
# print("Modelo guardado en", save_name)
# ```

# %% [markdown]
# ## 8. Observaciones y consejos finales
#
# 1. **Hyperparámetros**  
#    - `frames_per_batch` ≈ 1024–4096 (depende de tu GPU/CPU).  
#    - `num_iterations` = 200–300 (si quieres entrenar más tiempo, subirlo).  
#    - `ppo_epoch` = 3–6, `sub_batch_size` = 128.  
#    - `gamma` = 0.99, `lam` = 0.95, `clip_eps` = 0.2.  
#    - `learning_rate` = 2 × 10⁻⁴.  
#
# 2. **Funciones de TorchRL**  
#    - `ClipPPOLoss` engloba:  
#       • Cálculo del ratio πθ/πθ₀ con clipping.  
#       • Loss de valor (MSE) con coeficiente `value_loss_coeff`.  
#       • Término de entropía (`entropy_coeff`).  
#    - `GAE` crea las llaves `"advantage"` y `"return"` dentro del TensorDict.  
#    - `SyncDataCollector` con `frames_per_batch` devuelve un TensorDict con:
#       • `"rollout_info","reward_sum"`: suma de recompensas por episodio.  
#
# 3. **Gráficas**  
#    - Cada iteración `plot(logs)` reconstruye la figura desde cero (con `clear_output(wait=True)` y `display(fig)`),
#      de modo que verás en vivo cómo se actualizan las curvas de “Avg Reward (Train)” y “Avg Reward (Eval)”.  
#
# 4. **Exploración**  
#    - Usamos `set_exploration_type(ExplorationType.RANDOM)` durante el entrenamiento para que `ProbabilisticActor` samplee acciones.  
#    - Para evaluación, cambiamos a `ExplorationType.MODE` para usar la acción `argmax`.  
#
# 5. **Ajustes**  
#    - Si deseas ver el entorno “real” en evaluación, podrías crear un Gym env aparte con `render_mode="human"` y dibujar algunos episodios.  
#    - Para guardarlo periódicamente, inserta un bloque dentro del bucle:  
#      ```python
#      if iter_idx % 10 == 0:
#          torch.save(...).
#      ```  
#
# ———————————————————————————————————————————————————————————————————————————————
#
# ¡Listo! Ahora tienes el código completo con la corrección en `make_torchrl_env` para evitar el error de `seed` y `env_name`.  
